In [26]:
# removes annoying deprecation warnings 
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

import pandas as pd
from google.cloud import bigquery
from bq_helper import BigQueryHelper #third party library to translate google query data to dataframe
import matplotlib.pyplot as plt
import os, sys

sys.path.insert(0, './../utils/')

# custom files 
import random_forest_regressor as rfr
import utilities as util
import validation as cv
from sklearn.model_selection import RepeatedKFold

from sklearn.model_selection import train_test_split
from sklearn import cross_validation

%matplotlib inline

with open('../../key.txt') as f:
    content = f.readlines()

#add your own key here 
os.environ['GOOGLE_APPLICATION_CREDENTIALS']= content[0]

In [27]:
EPA_QUERY = """
        SELECT
            avg(CO_daily.arithmetic_mean) as COam,
            cast(EXTRACT(YEAR FROM CO_daily.date_local)*10000 +
            EXTRACT(MONTH FROM CO_daily.date_local)*100 + 
            EXTRACT(DAY FROM CO_daily.date_local) as string) as COdate,
            avg(PPM_daily.arithmetic_mean) as PPMam,
            EXTRACT(YEAR FROM PPM_daily.date_local)*10000 +
            EXTRACT(MONTH FROM PPM_daily.date_local)*100 + 
            EXTRACT(DAY FROM PPM_daily.date_local) as PPMdate,
            avg(SO2_daily.arithmetic_mean) as SO2am,
            EXTRACT(YEAR FROM SO2_daily.date_local)*10000 +
            EXTRACT(MONTH FROM SO2_daily.date_local)*100 + 
            EXTRACT(DAY FROM SO2_daily.date_local) as SO2date,
            avg(NO2_daily.arithmetic_mean) as NO2am,
            EXTRACT(YEAR FROM NO2_daily.date_local)*10000 +
            EXTRACT(MONTH FROM NO2_daily.date_local)*100 + 
            EXTRACT(DAY FROM NO2_daily.date_local) as NO2date
        FROM
          `bigquery-public-data.epa_historical_air_quality.co_daily_summary` as CO_daily
          INNER JOIN `bigquery-public-data.epa_historical_air_quality.pm25_frm_daily_summary` as PPM_daily ON CO_daily.date_local = PPM_daily.date_local
          INNER JOIN `bigquery-public-data.epa_historical_air_quality.so2_daily_summary` as SO2_daily ON CO_daily.date_local = SO2_daily.date_local
          INNER JOIN `bigquery-public-data.epa_historical_air_quality.no2_daily_summary` as NO2_daily ON CO_daily.date_local = NO2_daily.date_local
        WHERE CO_daily.state_name ="California" AND CO_daily.city_name="San Francisco" AND
        PPM_daily.state_name ="California" AND PPM_daily.city_name="San Francisco" AND
        SO2_daily.state_name ="California" AND SO2_daily.city_name="San Francisco" AND
        NO2_daily.state_name ="California" AND NO2_daily.city_name="San Francisco"
        GROUP BY COdate, PPMdate, SO2date, NO2date
        ORDER BY COdate DESC
        """
bq_assistant = BigQueryHelper("bigquery-public-data", "epa_historical_air_quality")
df_POLLUTION = bq_assistant.query_to_pandas(EPA_QUERY)

In [28]:
df_POLLUTION.head()

,COam,COdate,PPMam,PPMdate,SO2am,SO2date,NO2am,NO2date
0,0.410598,20081220,7.0,20081220,2.500000,20081220,19.450000,20081220
1,0.415399,20081217,6.5,20081217,1.669481,20081217,24.238095,20081217
2,0.327898,20081214,4.7,20081214,0.227922,20081214,16.608696,20081214
3,0.773279,20081211,22.1,20081211,2.962987,20081211,29.952381,20081211
4,0.618206,20081208,18.6,20081208,1.397403,20081208,22.863636,20081208


In [29]:
SF_CRIME_QUERY = """
        SELECT
            COUNT( DISTINCT unique_key) as count,
            cast(EXTRACT(YEAR FROM SFCrimeData.timestamp)*10000 +
            EXTRACT(MONTH FROM SFCrimeData.timestamp)*100 + 
            EXTRACT(DAY FROM SFCrimeData.timestamp) as string) as date
        FROM
          `bigquery-public-data.san_francisco_sfpd_incidents.sfpd_incidents` AS SFCrimeData
        WHERE category != "NON-CRIMINAL" AND category != "RECOVERED VEHICLE"
        GROUP BY date
        ORDER BY date DESC
        """

bq_assistant_SF_crime = BigQueryHelper("bigquery-public-data", "san_francisco_sfpd_incidents.sfpd_incidents")
df_SF_crime = bq_assistant_SF_crime.query_to_pandas(SF_CRIME_QUERY)

In [30]:
df_SF_census = pd.read_csv('../../data/censuspopulationsf.tsv', sep='\t', header=None)
df_SF_census.columns = ['year', 'pop']
df_SF_census.head(n=20)

,year,pop
0,2003,757638
1,2004,750133
2,2005,748846
3,2006,751431
4,2007,758348
5,2008,767067
6,2009,774347
7,2010,805770
8,2011,816294
9,2012,830406


In [31]:
# make column for counts per capita
util.per_capita(df_SF_crime, df_SF_census)
# merge CO and Crime data
df_POLLUTION.rename(columns={'COdate': 'date'}, inplace=True)
df_merged = util.merge_data(df_POLLUTION[['date', 'COam', 'PPMam', 'SO2am', 'NO2am']], df_SF_crime)
df_merged.head()

,date,COam,PPMam,SO2am,NO2am,count,per_capita
3432,20081220,0.410598,7.0,2.500000,19.450000,329,0.000429
3435,20081217,0.415399,6.5,1.669481,24.238095,266,0.000347
3438,20081214,0.327898,4.7,0.227922,16.608696,202,0.000263
3441,20081211,0.773279,22.1,2.962987,29.952381,277,0.000361
3444,20081208,0.618206,18.6,1.397403,22.863636,271,0.000353


In [32]:
# find optimum regressor
regr = rfr.find_regressor(df_merged[['date', 'COam', 'PPMam', 'SO2am', 'NO2am']].as_matrix(), df_merged['per_capita'].values)
regr

Fitting 3 folds for each of 10 candidates, totalling 30 fits


/home/kristina/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


[CV] n_estimators=250, min_samples_split=8, max_depth=50, bootstrap=True 
[CV] n_estimators=250, min_samples_split=8, max_depth=50, bootstrap=True 
[CV] n_estimators=250, min_samples_split=8, max_depth=50, bootstrap=True 
[CV] n_estimators=275, min_samples_split=4, max_depth=80, bootstrap=True 
[CV]  n_estimators=250, min_samples_split=8, max_depth=50, bootstrap=True, total=   0.3s
[CV] n_estimators=275, min_samples_split=4, max_depth=80, bootstrap=True 
[CV]  n_estimators=250, min_samples_split=8, max_depth=50, bootstrap=True, total=   0.4s
[CV] n_estimators=275, min_samples_split=4, max_depth=80, bootstrap=True 
[CV]  n_estimators=250, min_samples_split=8, max_depth=50, bootstrap=True, total=   0.4s
[CV] n_estimators=250, min_samples_split=2, max_depth=10, bootstrap=False 
[CV]  n_estimators=275, min_samples_split=4, max_depth=80, bootstrap=True, total=   0.4s
[CV] n_estimators=250, min_samples_split=2, max_depth=10, bootstrap=False 
[CV]  n_estimators=250, min_samples_split=2, max_d

[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    2.9s finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=70,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=4,
           min_weight_fraction_leaf=0.0, n_estimators=125, n_jobs=1,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [33]:
# split dataset 
X_train, X_test, y_train, y_test = train_test_split(df_merged[['date', 'COam', 'PPMam', 'SO2am', 'NO2am']], df_merged['per_capita'].values, test_size=0.33, random_state=42)
# make predictions based on optimum regressor
y_pred = rfr.fit_and_predict(regr, X_train, X_test, y_train, y_test)

./../utils/random_forest_regressor.py:18: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  regr.fit(X_train.as_matrix(), y_train)
./../utils/random_forest_regressor.py:19: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  return regr.predict(X_test.as_matrix())


In [34]:
cv.MSE(y_test, y_pred)
# leave one out cross validation 
# loo = cross_validation.LeaveOneOut(len(df_merged['per_capita'].values))
# loo_score = cv.Cross_Validation(loo, regr, df_merged[['date','am']].as_matrix(), df_merged['per_capita'].values)

1.7580211083441344e-09

In [35]:
# 10 fold tss cross validation
tss_score = cv.Cross_Validation(df_merged[['date', 'COam', 'PPMam', 'SO2am', 'NO2am']], df_merged['per_capita'], regr, 10)
print('10-fold cross validation using time series split (additive): {} '.format(tss_score))

10-fold cross validation using time series split (additive): 2.104379467336409e-09 


In [36]:
df_merged.corr()

,COam,PPMam,SO2am,NO2am,count,per_capita
COam,1.000000,0.509747,0.619965,0.802885,0.140665,0.145777
PPMam,0.509747,1.000000,0.608483,0.589155,0.034015,0.036684
SO2am,0.619965,0.608483,1.000000,0.726470,0.092678,0.098598
NO2am,0.802885,0.589155,0.726470,1.000000,0.148181,0.147951
count,0.140665,0.034015,0.092678,0.148181,1.000000,0.998241
per_capita,0.145777,0.036684,0.098598,0.147951,0.998241,1.000000
